<a href="https://colab.research.google.com/github/GoodchildTrevor/pack_hackaton/blob/main/Parser_Chunker_DRK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом коде я создаю текстовый csv файл следующейго содержания:
- данные из html файлов отдела DRK в виде текстовой базы данных

Вид:
- целевые столбцы Query, Answer, Department, Link, где
    
*   Query = названию файла html (по возможности без текстового мусора)
*   Answer содержит всю содержательную часть файлов папки DRK, соответствующую Query  в этой строке. Табличные данные вида вопрос-ответ были распарсены немного по-другому, строки таких таблиц стали отдельными строками в нашем csv, в этом случае создано несколько строк с одинаковым Query, соответствуюшим названию файла (Напрмер, Частые Вопросы), а внутри Answer будет лишь один смысловой вопрос-ответ, как отдельная смысловая еденица.
*   Department - отдел
*   Link - ссылка на конфлюенс






Загрузка необходимых библиотек. BeautifulSoup помогает распарсить данные из html файлов, os помогает работать с файлами, папками, переименованием, pandas - библиотека для обработки и анализа данных, fuzzywuzzy используется для нахождения наиболее похожих строк, может использоваться для нахождения опечаток

In [30]:
from bs4 import BeautifulSoup
import os
import pandas as pd
!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import process

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Задаю папку с файлами DRK

In [3]:
directory = '/content/drive/MyDrive/PEK/DRK/'

Создаю функцию, которая делит файлы папки на два типа - с таблицами вопросов и ответов, и без оных, они дальше будут обрабатывать по-разному.

In [39]:
def classify_html_files(directory):
    files_with_vopros = []
    files_without_vopros = []

    for filename in os.listdir(directory):
        if filename.endswith(".html"):
            if 'вопрос' in filename.lower() and 'задава' not in filename.lower():
                files_with_vopros.append(filename)
            else:
                files_without_vopros.append(filename)

    return files_with_vopros, files_without_vopros

files_with_vopros, files_without_vopros = classify_html_files(directory)

Инициализирую пустые дата фреймы для двух типов распарсеных файлов

In [41]:
# Create lists to hold DataFrames for tables and paragraphs
tables_dataframes = []
paragraphs_dataframes = []

Здесь происходит парсинг файлов с табличными данными с помощью инструментов BeautifulSoup. Строки таблиц html файла становятся строками датафрейма.

In [42]:
for filename in files_with_vopros:
    file_path = os.path.join(directory, filename)

    # Check if the file exists
    if os.path.exists(file_path) and filename.endswith('.html'):
        # Load the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract all tables
        tables = soup.find_all('table')

        # Convert each table to a pandas DataFrame
        for table_num, table in enumerate(tables):
            df = pd.read_html(str(table))[0]
            df['Query'] = os.path.splitext(filename)[0]  # Add a column to identify the source file without .html
            tables_dataframes.append(df)

Здесь происходит парсинг остальных файлов, опыть с помощью инструментов BeautifulSoup. Текстовое смысловое содежание одного html файла становится строкой датафрейма.

In [43]:
for filename in files_without_vopros:
    file_path = os.path.join(directory, filename)

    # Check if the file exists
    if os.path.exists(file_path) and filename.endswith('.html'):
        # Load the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract all paragraphs and concatenate their text
        paragraphs = soup.find_all('p')
        paragraphs_text = ' '.join(p.get_text(strip=True) for p in paragraphs)

        # Create a DataFrame for the concatenated paragraphs
        df_paragraphs = pd.DataFrame([[os.path.splitext(filename)[0], paragraphs_text]],
                                     columns=['Query', 'Answer'])

        # Append the DataFrame to the list
        paragraphs_dataframes.append(df_paragraphs)

Конкатенируем все строки таличного парсинга в дата фрейм combined_tables_df, всте строки парсинга остальных файлов в дата фрейм combined_paragrphs_df

In [44]:
combined_tables_df = pd.concat(tables_dataframes, ignore_index=True)
combined_paragraphs_df = pd.concat(paragraphs_dataframes, ignore_index=True)

In [45]:
display(combined_tables_df)

,0,1,Query
0,Вопрос,Ответ,Вопросы по грузам с наложенным платежом
1,Возможно ли сделать корректировку плательщика ...,Нет. Услуги (ТЭУ + страхование) возможно скорр...,Вопросы по грузам с наложенным платежом
2,"Клиенту необходим осмотр груза, а груз в ЗТУ. ...",Процесс передачи груза получателю аналогичен п...,Вопросы по грузам с наложенным платежом
3,"Действия сотрудников, если клиент не согласен ...",Услуги могут быть включены в стоимость НП в то...,Вопросы по грузам с наложенным платежом
4,"Какие действия сотрудника, если в 1С указано т...","Это бывает у грузов ИЗИ УЭЙ, у них упрощенная ...",Вопросы по грузам с наложенным платежом
...,...,...,...
81,"Все условия выполняются, но баллы не начислены...",Перевод на 1906 или самостоятельно направьте з...,Вопросы по личному кабинету - База знаний ДРК...
82,Как делать смену ГП в ЛК,"Вам необходимо зайти в раздел ""Мои грузы"", выб...",Вопросы по личному кабинету - База знаний ДРК...
83,Как делать перенаправку в ЛК,"Вам необходимо зайти в раздел ""Мои грузы"", выб...",Вопросы по личному кабинету - База знаний ДРК...
84,Как делать приостановку в ЛК,"Вам необходимо зайти в раздел ""Мои грузы"", выб...",Вопросы по личному кабинету - База знаний ДРК...


Теперь мы сшиваем вопрос и ответ в одну ячейку в combined_tables_df

In [46]:
combined_tables_df = combined_tables_df.drop(index=0)
combined_tables_df['Answer'] = combined_tables_df[0] + ' ' + combined_tables_df[1]
combined_tables_df = combined_tables_df.drop([0, 1], axis=1)

In [47]:
display(combined_tables_df)

,Query,Answer
1,Вопросы по грузам с наложенным платежом,Возможно ли сделать корректировку плательщика ...
2,Вопросы по грузам с наложенным платежом,"Клиенту необходим осмотр груза, а груз в ЗТУ. ..."
3,Вопросы по грузам с наложенным платежом,"Действия сотрудников, если клиент не согласен ..."
4,Вопросы по грузам с наложенным платежом,"Какие действия сотрудника, если в 1С указано т..."
5,Вопросы по грузам с наложенным платежом,"После оплаты НП и получения груза выяснилось, ..."
...,...,...
81,Вопросы по личному кабинету - База знаний ДРК...,"Все условия выполняются, но баллы не начислены..."
82,Вопросы по личному кабинету - База знаний ДРК...,Как делать смену ГП в ЛК Вам необходимо зайти ...
83,Вопросы по личному кабинету - База знаний ДРК...,Как делать перенаправку в ЛК Вам необходимо за...
84,Вопросы по личному кабинету - База знаний ДРК...,Как делать приостановку в ЛК Вам необходимо за...


Убираем шумовое окончание некоторых файлов, чтобы оставить только смысл

In [48]:
combined_tables_df['Query'] = combined_tables_df['Query'].str.replace(' - База знаний ДРК - База знаний', '')
display(combined_tables_df)

,Query,Answer
1,Вопросы по грузам с наложенным платежом,Возможно ли сделать корректировку плательщика ...
2,Вопросы по грузам с наложенным платежом,"Клиенту необходим осмотр груза, а груз в ЗТУ. ..."
3,Вопросы по грузам с наложенным платежом,"Действия сотрудников, если клиент не согласен ..."
4,Вопросы по грузам с наложенным платежом,"Какие действия сотрудника, если в 1С указано т..."
5,Вопросы по грузам с наложенным платежом,"После оплаты НП и получения груза выяснилось, ..."
...,...,...
81,Вопросы по личному кабинету,"Все условия выполняются, но баллы не начислены..."
82,Вопросы по личному кабинету,Как делать смену ГП в ЛК Вам необходимо зайти ...
83,Вопросы по личному кабинету,Как делать перенаправку в ЛК Вам необходимо за...
84,Вопросы по личному кабинету,Как делать приостановку в ЛК Вам необходимо за...


In [49]:
combined_tables_df['Query'] = combined_tables_df['Query'].str.replace(' - База знаний', '')

display(combined_tables_df)

,Query,Answer
1,Вопросы по грузам с наложенным платежом,Возможно ли сделать корректировку плательщика ...
2,Вопросы по грузам с наложенным платежом,"Клиенту необходим осмотр груза, а груз в ЗТУ. ..."
3,Вопросы по грузам с наложенным платежом,"Действия сотрудников, если клиент не согласен ..."
4,Вопросы по грузам с наложенным платежом,"Какие действия сотрудника, если в 1С указано т..."
5,Вопросы по грузам с наложенным платежом,"После оплаты НП и получения груза выяснилось, ..."
...,...,...
81,Вопросы по личному кабинету,"Все условия выполняются, но баллы не начислены..."
82,Вопросы по личному кабинету,Как делать смену ГП в ЛК Вам необходимо зайти ...
83,Вопросы по личному кабинету,Как делать перенаправку в ЛК Вам необходимо за...
84,Вопросы по личному кабинету,Как делать приостановку в ЛК Вам необходимо за...


In [50]:
combined_paragraphs_df['Query'] = combined_paragraphs_df['Query'].str.replace(' - База знаний ДРК', '')
display(combined_paragraphs_df)

,Query,Answer
0,Топливная надбавка_топливный сбор в ПЭ_ЭР,С 25.03.2024 г. во всех печатных формах ПЭ/ЭР ...
1,Неоформленные грузы,"Проверьте наличие заявки на забор груза, незав..."
2,Выдача грузов по СМС при доставке,Выдача груза по СМС при доставке возможна в сл...
3,Клиент просит отключить уведомления (E-mail; В...,"Для отключения уведомлений, необходимо уточни..."
4,Интернет-Эквайринг (оплата банковской картои...,На главной странице сайтаwww.pecom.ruклиенту ...
...,...,...
287,Консультация по грузам с забором_доставкой - ...,Первичное планированиезаявок на забор/доставку...
288,Прием_выдача грузов в ПВЗ (пункт выдачи заказо...,"На сайте в ""Адреса"" можно посмотреть информац..."
289,Нештатные ситуации при доставке грузов на ПВЗ ...,"Существует ряд причин, по которым грузы воврем..."
290,Спец.условие _Отказ от ЗТУ_ - База знаний,Отказ от ЗТУ сами НЕ предлагаем! Возможность ...


In [51]:
combined_paragraphs_df['Query'] = combined_paragraphs_df['Query'].str.replace(' - База знаний', '')
display(combined_paragraphs_df)

,Query,Answer
0,Топливная надбавка_топливный сбор в ПЭ_ЭР,С 25.03.2024 г. во всех печатных формах ПЭ/ЭР ...
1,Неоформленные грузы,"Проверьте наличие заявки на забор груза, незав..."
2,Выдача грузов по СМС при доставке,Выдача груза по СМС при доставке возможна в сл...
3,Клиент просит отключить уведомления (E-mail; В...,"Для отключения уведомлений, необходимо уточни..."
4,Интернет-Эквайринг (оплата банковской картои...,На главной странице сайтаwww.pecom.ruклиенту ...
...,...,...
287,Консультация по грузам с забором_доставкой,Первичное планированиезаявок на забор/доставку...
288,Прием_выдача грузов в ПВЗ (пункт выдачи заказов),"На сайте в ""Адреса"" можно посмотреть информац..."
289,Нештатные ситуации при доставке грузов на ПВЗ,"Существует ряд причин, по которым грузы воврем..."
290,Спец.условие _Отказ от ЗТУ_,Отказ от ЗТУ сами НЕ предлагаем! Возможность ...


Соединяем оба дата фрейма теперь в один

In [52]:
concatenated_drk_df = pd.concat([combined_tables_df, combined_paragraphs_df], axis=0).reset_index(drop=True)

In [53]:
display(concatenated_drk_df)

,Query,Answer
0,Вопросы по грузам с наложенным платежом,Возможно ли сделать корректировку плательщика ...
1,Вопросы по грузам с наложенным платежом,"Клиенту необходим осмотр груза, а груз в ЗТУ. ..."
2,Вопросы по грузам с наложенным платежом,"Действия сотрудников, если клиент не согласен ..."
3,Вопросы по грузам с наложенным платежом,"Какие действия сотрудника, если в 1С указано т..."
4,Вопросы по грузам с наложенным платежом,"После оплаты НП и получения груза выяснилось, ..."
...,...,...
372,Консультация по грузам с забором_доставкой,Первичное планированиезаявок на забор/доставку...
373,Прием_выдача грузов в ПВЗ (пункт выдачи заказов),"На сайте в ""Адреса"" можно посмотреть информац..."
374,Нештатные ситуации при доставке грузов на ПВЗ,"Существует ряд причин, по которым грузы воврем..."
375,Спец.условие _Отказ от ЗТУ_,Отказ от ЗТУ сами НЕ предлагаем! Возможность ...


Убираем выявленное шумовое окончание из столбца ответов

In [54]:
# Text to be removed
text_to_remove = "Добавить комментарий ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня."
concatenated_drk_df['Answer'] = concatenated_drk_df['Answer'].str.replace(text_to_remove, '')

In [55]:
display(concatenated_drk_df)

,Query,Answer
0,Вопросы по грузам с наложенным платежом,Возможно ли сделать корректировку плательщика ...
1,Вопросы по грузам с наложенным платежом,"Клиенту необходим осмотр груза, а груз в ЗТУ. ..."
2,Вопросы по грузам с наложенным платежом,"Действия сотрудников, если клиент не согласен ..."
3,Вопросы по грузам с наложенным платежом,"Какие действия сотрудника, если в 1С указано т..."
4,Вопросы по грузам с наложенным платежом,"После оплаты НП и получения груза выяснилось, ..."
...,...,...
372,Консультация по грузам с забором_доставкой,Первичное планированиезаявок на забор/доставку...
373,Прием_выдача грузов в ПВЗ (пункт выдачи заказов),"На сайте в ""Адреса"" можно посмотреть информац..."
374,Нештатные ситуации при доставке грузов на ПВЗ,"Существует ряд причин, по которым грузы воврем..."
375,Спец.условие _Отказ от ЗТУ_,Отказ от ЗТУ сами НЕ предлагаем! Возможность ...


Считываем текстовый файл со ссылками, чтобы присоединить их к срокам дата фрейма

In [56]:
links=pd.read_csv('/content/drive/MyDrive/PEK/DRK.txt', sep='/t', header=None)
display(links) #links

<ipython-input-56-6ec837c8420e>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  links=pd.read_csv('/content/drive/MyDrive/PEK/DRK.txt', sep='/t', header=None)


,0
0,Топливная надбавка_топливный сбор в ПЭ_ЭР;http...
1,Неоформленные грузы;http://confluence/pages/vi...
2,Выдача грузов по СМС при доставке;http://confl...
3,Клиент просит отключить уведомления (E-mail; В...
4,Интернет-Эквайринг (оплата банковской картой н...
...,...
295,СКРИПТЫ разговоров с клиентами;http://confluen...
296,Консультация по грузам с забором/доставкой;htt...
297,Прием/выдача грузов в ПВЗ (пункт выдачи заказо...
298,Нештатные ситуации при доставке грузов на ПВЗ;...


Разделяем строки текстового файла на две колокни File - название файла в папке и Link - ссылка

In [57]:
# Split the unnamed column by the last ";"

# Function to split by the last semicolon
def split_by_last_semicolon(text):
    parts = text.rsplit(';', 1)
    return parts if len(parts) == 2 else [text, '']

# Apply the function to split the unnamed column
links_split = links.iloc[:, 0].apply(split_by_last_semicolon).apply(pd.Series)

# Rename the columns
links_split.columns = ['File', 'Link']

display(links_split)

,File,Link
0,Топливная надбавка_топливный сбор в ПЭ_ЭР,http://confluence/pages/viewpage.action?pageId...
1,Неоформленные грузы,http://confluence/pages/viewpage.action?pageId...
2,Выдача грузов по СМС при доставке,http://confluence/pages/viewpage.action?pageId...
3,Клиент просит отключить уведомления (E-mail; В...,http://confluence/pages/viewpage.action?pageId...
4,Интернет-Эквайринг (оплата банковской картой н...,http://confluence/pages/viewpage.action?pageId...
...,...,...
295,СКРИПТЫ разговоров с клиентами,http://confluence/pages/viewpage.action?pageId...
296,Консультация по грузам с забором/доставкой,http://confluence/pages/viewpage.action?pageId...
297,Прием/выдача грузов в ПВЗ (пункт выдачи заказов),http://confluence/pages/viewpage.action?pageId...
298,Нештатные ситуации при доставке грузов на ПВЗ,http://confluence/pages/viewpage.action?pageId...


Значения в поле File дата фрейма links_split не всегда вточности сопадают с полем Query основного фрейма. Поэтому я делаю функцию поиска максимальной похожести с использованием библиотеки fuzzywuzzy

In [58]:
def get_best_match(query, choices, threshold=80):
    result = process.extractOne(query, choices)
    if result:
        best_match = result[0]
        score = result[1]
        if score >= threshold:
            return best_match
    return None

In [59]:
links_split['Best_Match'] = links_split['File'].apply(lambda x: get_best_match(x, concatenated_drk_df['Query']))

Quality Control что все нашлось корректно

In [60]:
display(links_split)

,File,Link,Best_Match
0,Топливная надбавка_топливный сбор в ПЭ_ЭР,http://confluence/pages/viewpage.action?pageId...,Топливная надбавка_топливный сбор в ПЭ_ЭР
1,Неоформленные грузы,http://confluence/pages/viewpage.action?pageId...,Неоформленные грузы
2,Выдача грузов по СМС при доставке,http://confluence/pages/viewpage.action?pageId...,Выдача грузов по СМС при доставке
3,Клиент просит отключить уведомления (E-mail; В...,http://confluence/pages/viewpage.action?pageId...,Клиент просит отключить уведомления (E-mail; В...
4,Интернет-Эквайринг (оплата банковской картой н...,http://confluence/pages/viewpage.action?pageId...,Интернет-Эквайринг (оплата банковской картои...
...,...,...,...
295,СКРИПТЫ разговоров с клиентами,http://confluence/pages/viewpage.action?pageId...,СКРИПТЫ разговоров с клиентами
296,Консультация по грузам с забором/доставкой,http://confluence/pages/viewpage.action?pageId...,Консультация по грузам с забором_доставкой
297,Прием/выдача грузов в ПВЗ (пункт выдачи заказов),http://confluence/pages/viewpage.action?pageId...,Прием_выдача грузов в ПВЗ (пункт выдачи заказов)
298,Нештатные ситуации при доставке грузов на ПВЗ,http://confluence/pages/viewpage.action?pageId...,Нештатные ситуации при доставке грузов на ПВЗ


Одно название файла определилось неверно, наверное дело в очень большой длине названия, корректирую его руками, чтоы ссылка в ответах отображалась правильно

In [61]:
links_split.at[49, 'Best_Match'] = 'Можно ли принять доверенность, если она подписана новым Генеральны~о клиент принес Протокол о назначении и Приказ на нового директора_'

In [62]:
display(links_split)

,File,Link,Best_Match
0,Топливная надбавка_топливный сбор в ПЭ_ЭР,http://confluence/pages/viewpage.action?pageId...,Топливная надбавка_топливный сбор в ПЭ_ЭР
1,Неоформленные грузы,http://confluence/pages/viewpage.action?pageId...,Неоформленные грузы
2,Выдача грузов по СМС при доставке,http://confluence/pages/viewpage.action?pageId...,Выдача грузов по СМС при доставке
3,Клиент просит отключить уведомления (E-mail; В...,http://confluence/pages/viewpage.action?pageId...,Клиент просит отключить уведомления (E-mail; В...
4,Интернет-Эквайринг (оплата банковской картой н...,http://confluence/pages/viewpage.action?pageId...,Интернет-Эквайринг (оплата банковской картои...
...,...,...,...
295,СКРИПТЫ разговоров с клиентами,http://confluence/pages/viewpage.action?pageId...,СКРИПТЫ разговоров с клиентами
296,Консультация по грузам с забором/доставкой,http://confluence/pages/viewpage.action?pageId...,Консультация по грузам с забором_доставкой
297,Прием/выдача грузов в ПВЗ (пункт выдачи заказов),http://confluence/pages/viewpage.action?pageId...,Прием_выдача грузов в ПВЗ (пункт выдачи заказов)
298,Нештатные ситуации при доставке грузов на ПВЗ,http://confluence/pages/viewpage.action?pageId...,Нештатные ситуации при доставке грузов на ПВЗ


Теперь присоединяем ссылки к дата фрейму с распарсенными html

In [64]:
merged_df=[]
# Create a function to merge based on the custom condition
def custom_merge(df1, df2):
    merged_data = []
    for _, row1 in df1.iterrows():
        query = row1['Query']
        matching_link = None
        for _, row2 in df2.iterrows():
            if query.startswith(row2['Best_Match']):
                matching_link = row2['Link']
                break
        merged_data.append({
            'Query': row1['Query'],
            'Answer': row1['Answer'],
            'Link': matching_link
        })
    return pd.DataFrame(merged_data)

# Merge DataFrames using the custom function
merged_drk_df = custom_merge(concatenated_drk_df, links_split)

Обращаю внимание, что два html файла были без ссылок на конфлюенс:

*   Инструкция по работе с ККТ, подключенной к СПО _Пегас_ - Пользовательская документация ИС
*   Частые Вопросы - Пользовательская документация ИС



Присоединяю к фрейму колонку с названием отдела

In [66]:
merged_drk_df['Department'] = 'DRK'

In [67]:
merged_drk_df.to_csv('/content/drive/MyDrive/merged_drk_df.csv', index=False)